In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
emails = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/input/emails.csv')
# email_subset = emails[:10000]
email_subset = emails.sample(frac=0.02, random_state=1)

print(email_subset.shape)
print(email_subset.head())

(10348, 2)
                              file  \
186822  jones-t/all_documents/634.   
308790  mann-k/all_documents/5690.   
82383         dasovich-j/sent/423.   
227299          kaminski-v/var/63.   
301824     mann-k/_sent_mail/3208.   

                                                  message  
186822  Message-ID: <17820178.1075846925335.JavaMail.e...  
308790  Message-ID: <29110382.1075845717882.JavaMail.e...  
82383   Message-ID: <6812040.1075843194135.JavaMail.ev...  
227299  Message-ID: <21547648.1075856642126.JavaMail.e...  
301824  Message-ID: <12684200.1075846107179.JavaMail.e...  


In [ ]:
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

In [ ]:
def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        'to': map_to_list(emails, 'to'),
        'from_': map_to_list(emails, 'from')
    }


In [ ]:
def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results

In [ ]:
email_df = pd.DataFrame(parse_into_emails(email_subset.message))
print(email_df.head())

                                                body  \
0  It would be nice if you could be at my dinner,...   
1  Absolutely.Good point!  Can Peter start to dra...   
2  My apologies.  My schedule melted down after w...   
3  Vince,UK VAR breached the limit last week.UK t...   
4  Any problems/comments?AM ---------------------...   

                                                  to  \
0                           alicia.goodrow@enron.com   
1                          Kay Mann/Corp/Enron@ENRON   
2                        christine.piesco@oracle.com   
3  Richard Lewis/LON/ECT@ECT, James New/LON/ECT@E...   
4  Don Hammond/PDX/ECT@ECT, Jody Blackburn/PDX/EC...   

                               from_  
0               tana.jones@enron.com  
1  Sheila Tweed@ECT on 05/15/2001 06  
2            jeff.dasovich@enron.com  
3        tanya.tamarchenko@enron.com  
4                 kay.mann@enron.com  


In [ ]:
import re
import numpy as np

In [ ]:
!python --version


Python 3.10.12


In [ ]:
#!pip install gensim==3.8.3

In [ ]:
import gensim

In [ ]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
# spacy for lemmatization
import spacy

In [ ]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 28.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicor

In [ ]:
# for plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [ ]:
 import nltk
 nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# prep NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(email_df.iloc[2]['body']) # displays info below

My apologies.  My schedule melted down after we talked on Monday.  Here'swhere folks came out.  There's some concern about size.  We're supposed to beno larger than 3, but we lobbied Aceves and he apparently Ok'd our"oversized" group.  The other folks in the group--who talked to himoriginally--are pretty sure that five will violate the rules.  Folks wonderedif there were other groups that are smaller than ours that you could hook upwith.  Sorry about that---it's a wrinkle that I didn't think about when wespoke.  If it gets real ugly trying to find a smaller group, let me know.Fortunately there's not another team case due for two weeks.Best,Jeff


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Convert email body to list
data = email_df.body.values.tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data_words = list(sent_to_words(data))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(data_words[3])

['vince', 'uk', 'var', 'breached', 'the', 'limit', 'last', 'week', 'uk', 'traders', 'asked', 'us', 'to', 'review', 'the', 'correlations', 'across', 'uk', 'gas', 'and', 'power', 'aswell', 'as', 'the', 'correlations', 'across', 'efa', 'slots', 'we', 'did', 'part', 'of', 'the', 'work', 'last', 'week', 'now', 'we', 'll', 'update', 'the', 'correlations', 'based', 'on', 'historical', 'prices', 'tanya', 'richard', 'lewisleppard', 'lon', 'ect', 'ect', 'rudy', 'dautel', 'hou', 'ect', 'ect', 'kirstee', 'hewitt', 'lon', 'ect', 'ect', 'naveen', 'andrews', 'corp', 'enron', 'enron', 'david', 'port', 'market', 'risk', 'corp', 'enron', 'enron', 'tedmurphy', 'hou', 'ect', 'ect', 'simon', 'hastings', 'lon', 'ect', 'ect', 'paul', 'arcy', 'lon', 'ect', 'ect', 'amirghodsian', 'lon', 'ect', 'ectthanks', 'tanya', 'these', 'are', 'interesting', 'results', 'am', 'on', 'vacation', 'next', 'week', 'sohere', 'are', 'my', 'current', 'thoughts', 'am', 'contactable', 'on', 'my', 'mobile', 'if', 'necessary', 'gas', '

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from gensim.models.phrases import Phrases, Phraser

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[200]]])

['image_image_image', 'getting', 'together', 'for', 'the', 'holidays', 'is', 'something', 'we', 'all', 'enjoy', 'whether', 'it', 'gathering', 'with', 'old', 'friends', 'from', 'out', 'of', 'town', 'or', 'hanging', 'out', 'with', 'the', 'usual', 'gang', 'every', 'gathering', 'this', 'time', 'of', 'year', 'seems', 'little', 'more', 'special', 'we', 'at', 'miller_brewing', 'wish', 'you', 'many', 'happy', 'celebrations', 'this', 'season', 'and', 'thank', 'you', 'for', 'enjoying', 'those', 'occasions', 'responsibly', 'happy_holidays', 'image', 'this', 'mail', 'is', 'not', 'sent', 'unsolicited', 'you', 'subscribed', 'to', 'receive', 'information', 'from', 'miller_brewing', 'at', 'miller', 'web_site', 'or', 'event', 'must', 'be', 'or', 'older', 'to', 'visit', 'our', 'web_site', 'miller_brewing', 'co', 'milwaukee', 'wi', 'miller_brewing', 'company', 'milwaukee', 'wi', 'privacy_statement', 'image_image_image_image', 'this', 'message', 'was', 'sent', 'by', 'miller_brewing', 'company', 'using', '

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#python -m spacy download en

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(data_lemmatized[200])

['image_image', 'image', 'get', 'together', 'holiday', 'enjoy', 'gather', 'old', 'friend', 'town', 'hang', 'usual', 'gang', 'gathering', 'time', 'year', 'seem', 'little', 'special', 'miller_brewing', 'wish', 'many', 'happy', 'celebration', 'season', 'thank', 'enjoy', 'occasion', 'responsibly', 'image', 'mail', 'send', 'unsolicited', 'subscribed', 'receive', 'information', 'miller_brewe', 'event', 'old', 'visit', 'miller_brewe', 'miller_brewe', 'company', 'privacy', 'statement', 'image_image', 'image_image', 'message', 'send', 'miller_brewe', 'company', 'use', 'click', 'prefer', 'receive', 'future', 'mail', 'miller_brewe', 'company', 'click', 'view', 'permission', 'marketing', 'policy', 'image']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# create dictionary and corpus both are needed for (LDA) topic modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus_emails = [id2word.doc2bow(text) for text in texts]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#!unzip mallet-2.0.8.zip

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
mallet_path = './mallet-2.0.8/bin/mallet'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_emails,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Get topics for each email
email_topics = [lda_model.get_document_topics(corpus_email) for corpus_email in corpus_emails]

In [ ]:
email_topics

[[(2, 0.022257898),
  (3, 0.1015011),
  (7, 0.013093097),
  (9, 0.049809523),
  (14, 0.011117831),
  (18, 0.2885223),
  (19, 0.04640571),
  (21, 0.2999896),
  (22, 0.06345747),
  (25, 0.011348495),
  (27, 0.02336284)],
 [(2, 0.020365786),
  (3, 0.08036371),
  (7, 0.012055064),
  (9, 0.23333578),
  (14, 0.010178101),
  (19, 0.042683583),
  (21, 0.22095631),
  (22, 0.062106904),
  (23, 0.045135118),
  (25, 0.13806033),
  (27, 0.021468025),
  (28, 0.05046883)],
 [(2, 0.010289254),
  (3, 0.114443764),
  (9, 0.0529877),
  (18, 0.025791258),
  (19, 0.089212075),
  (21, 0.5706853),
  (22, 0.061472166),
  (25, 0.021056315),
  (27, 0.010800163)],
 [(3, 0.048526153),
  (6, 0.010333092),
  (7, 0.07985647),
  (9, 0.41027808),
  (10, 0.05230416),
  (11, 0.12467675),
  (19, 0.10564691),
  (21, 0.0789712),
  (22, 0.0338407),
  (28, 0.03228021)],
 [(3, 0.19419657),
  (9, 0.10657093),
  (12, 0.30419958),
  (14, 0.03785499),
  (19, 0.03967352),
  (21, 0.03438014),
  (22, 0.21387893),
  (27, 0.027048048)

In [ ]:
num_topics=20
email_keywords = [[word for word, prob in lda_model.show_topic(topic, topn=5)] for topic in range(num_topics)]

In [ ]:
email_keywords

[['appear', 'ensure', 'correct', 'arrangement', 'thus'],
 ['employee', 'write', 'enron', 'bankruptcy', 'fund'],
 ['com', 'enron', 'hotmail', 'net', 'priority'],
 ['thank', 'call', 'know', 'need', 'question'],
 ['award', 'investigation', 'publication', 'primarily', 'instance'],
 ['click', 'way', 'sport', 'free', 'online'],
 ['book', 'volume', 'check', 'head', 'counterparty'],
 ['say', 'power', 'energy', 'state', 'electricity'],
 ['df_forwarde', 'layniex', 'amendthe', 'uponpatent', 'multibranch'],
 ['change', 'update', 'contract', 'start', 'schedule'],
 ['afternoon', 'curve', 'vince', 'fuel', 'objective'],
 ['ect', 'hou', 'enronxgate', 'enron', 'lon'],
 ['agreement', 'document', 'file', 'appropriate', 'catch'],
 ['bill', 'governor', 'vote', 'legislature', 'amend'],
 ['mail', 'price', 'rate', 'transaction', 'section'],
 ['phillip', 'wereinformation', 'uponpatent', 'thesurrounding', 'amendthe'],
 ['player', 'hopefully', 'tonight', 'youcan', 'wereinformation'],
 ['game', 'commissioner', 'ca

In [ ]:
# topic modeling
# corpus, dictionary and number of topics required for LDA
# alpha and eta are hyperparameters that affect sparsity of the topics
# chunksize is the number of documents to be used in each training chunk
# update_every determines how often the model parameters should be updated
# passes is the total number of training passes
# Print the Keyword in the 10 topics

In [ ]:
print(lda_model.print_topics())# The weights reflect how important a keyword is to that topic.

[(8, '0.015*"df_forwarde" + 0.000*"uponpatent" + 0.000*"voice_mail" + 0.000*"layniex" + 0.000*"multibranch" + 0.000*"amendthe" + 0.000*"wereinformation" + 0.000*"thesurrounding" + 0.000*"townuntil" + 0.000*"resourceswa"'), (15, '0.035*"phillip" + 0.000*"thesurrounding" + 0.000*"tenantsclaim" + 0.000*"multibranch" + 0.000*"amendthe" + 0.000*"wereinformation" + 0.000*"uponpatent" + 0.000*"townuntil" + 0.000*"voice_mail" + 0.000*"propertyclaim"'), (29, '0.038*"sara" + 0.034*"ship" + 0.008*"lateral" + 0.000*"thesurrounding" + 0.000*"multibranch" + 0.000*"amendthe" + 0.000*"wereinformation" + 0.000*"uponpatent" + 0.000*"townuntil" + 0.000*"resourceswa"'), (17, '0.208*"game" + 0.057*"commissioner" + 0.045*"category" + 0.013*"defer" + 0.003*"hurt" + 0.000*"multibranch" + 0.000*"amendthe" + 0.000*"wereinformation" + 0.000*"uponpatent" + 0.000*"layniex"'), (16, '0.095*"player" + 0.050*"hopefully" + 0.049*"tonight" + 0.004*"youcan" + 0.000*"amendthe" + 0.000*"wereinformation" + 0.000*"uponpatent

In [ ]:
doc_lda = lda_model[corpus_emails]

In [ ]:
# Model perplexity and topic coherence provide a convenient
# measure to judge how good a given topic model is.
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus_emails))  # a measure of how good the model is. lower the better.



Perplexity:  -17.560575195589756


In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.389409060206401


In [ ]:
!pip install pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 54.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.3
    Uninstalling pandas-2.1.3:
      Successfully uninstalled pandas-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(lda_model, corpus_emails, id2word)

In [ ]:
pyLDAvis.display(vis)

In [ ]:
# now using mallet

In [ ]:
#mallet_path = 'mallet-2.0.8/bin/mallet'

In [ ]:
#!pip install gensim==3.8.3

In [ ]:
#from gensim.models.wrappers import LdaMallet
#from gensim.models import LdaMallet

#ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
#ldamallet = LdaMallet(mallet_path, corpus=corpus_emails, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
#print(ldamallet.show_topics(formatted=False))

In [ ]:
# Compute Coherence Score
#coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
#coherence_ldamallet = coherence_model_ldamallet.get_coherence()
#print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
# # Visualize the topics with mallet model
# pyLDAvis.enable_notebook(sort=True)
# vis = pyLDAvis.gensim.prepare(ldamallet, corpus, id2word)
# pyLDAvis.display(vis)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# run
#model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus_emails, texts=data_lemmatized, start=2, limit=40, step=6)


In [ ]:
# Show graph
#limit=40; start=2; step=6;
#x = range(start, limit, step)
#plt.plot(x, coherence_values)
#plt.xlabel("Num Topics")
#plt.ylabel("Coherence score")
#plt.legend(("coherence_values"), loc='best')
#plt.show()

In [ ]:
# Print the coherence scores
#for m, cv in zip(x, coherence_values):
#    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
#optimal_model = model_list[4]
#model_topics = optimal_model.show_topics(formatted=False)
#print(optimal_model.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus_emails, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus_emails]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)



In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus_emails, texts=data)

Streaming output truncated to the last 5000 lines.
<ipython-input-56-372d484f8be9>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
<ipython-input-56-372d484f8be9>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
<ipython-input-56-372d484f8be9>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
<ipython-input-56-372d484f8be9>:14: FutureWarning: The frame.app

In [ ]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [ ]:
# Show
df_dominant_topic.head(100)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,21,0.3000,"get, go, time, good, see, think, week, day, ma...","It would be nice if you could be at my dinner,..."
1,1,9,0.2333,"change, update, contract, start, schedule, opt...",Absolutely.Good point! Can Peter start to dra...
2,2,21,0.5707,"get, go, time, good, see, think, week, day, ma...",My apologies. My schedule melted down after w...
3,3,9,0.4103,"change, update, contract, start, schedule, opt...","Vince,UK VAR breached the limit last week.UK t..."
4,4,12,0.3042,"agreement, document, file, appropriate, catch,...",Any problems/comments?AM ---------------------...
...,...,...,...,...,...
95,95,2,0.4281,"com, enron, hotmail, net, priority, complease,...","dporter3@enron.com, eric.linder@enron.com, gei..."
96,96,3,0.2270,"thank, call, know, need, question, send, let, ...","elaine.concklin@enron.com, john.keiser@enron.c..."
97,97,21,0.3197,"get, go, time, good, see, think, week, day, ma...",michelle.lokay@enron.comAfter reviewing the li...
98,98,21,0.3343,"get, go, time, good, see, think, week, day, ma...","Yes, i can make a little money on the long sid..."


In [ ]:
df_dominant_topic.Keywords.iloc[1]

'change, update, contract, start, schedule, option, draft, date, form, position'

In [ ]:
df_dominant_topic.Text.iloc[1]

'Absolutely.Good point!  Can Peter start to draft an override letter?Kay Mann@ENRONDieball/ENRON_DEVELOPMENT@ENRON_DEVELOPMENt, John GRigby/ENRON_DEVELOPMENT@ENRON_DEVELOPMENTAs a reminder to all of us, we will need a form override letter to go withthe form turbine contract.Kay'

In [ ]:
df_dominant_topic.Keywords.iloc[10]

'thank, call, know, need, question, send, let, forward, regard, review'

In [ ]:
df_dominant_topic.Text.iloc[10]

'Lisa,Thanks.  I forwarded Roger the docs separately, but thanks for the reminder.I am capable of forgetting!KayLisa BillsKay,1.  Finance will have comments to you this afternoon.2.  I noticed just now that Roger Ondreko, as accounting signoff, was notcopied.  Please insure he receives a set of agreements and comments timely sowe can get the package out to the bank on the time line you require.Thanks, Lisa'

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)],
                                            axis=0)

In [ ]:
df_topic_sents_keywords[df_topic_sents_keywords['Dominant_Topic']==14]


,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
75,14,0.1832,"mail, price, rate, transaction, section, amoun...",T Paul Johnson@ENRONA number of pieces of on g...
134,14,0.2214,"mail, price, rate, transaction, section, amoun...",rich.heidorn@ipgdirect.comAug.-Nov.marjorie.ad...
176,14,0.1789,"mail, price, rate, transaction, section, amoun...",Markets and 21 December 2001 Day-Ahead MarketT...
184,14,0.2194,"mail, price, rate, transaction, section, amoun...",BUY 25 OPVSPT at 83.0000 - SALT RIVER PROJE...
677,14,0.2159,"mail, price, rate, transaction, section, amoun...","Dawn,NGI-SOCALNGI-PGE/CGDidn't you used to wor..."
935,14,0.3543,"mail, price, rate, transaction, section, amoun...",---------------------- Forwarded by Evelyn Met...
1074,14,0.1630,"mail, price, rate, transaction, section, amoun...","As of today, Power Gas Marketing volume has de..."
1142,14,0.3180,"mail, price, rate, transaction, section, amoun...","No, I think this one is for you...Marcus Nette..."
1315,14,0.2686,"mail, price, rate, transaction, section, amoun...",Mike Swerzbindeal 517384Prebon shows price as ...
1334,14,0.3132,"mail, price, rate, transaction, section, amoun...",---------------------- Forwarded by Darron C G...


In [ ]:
# Reset Index
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

In [ ]:
# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

In [ ]:
# Show
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0,0.1916,"appear, ensure, correct, arrangement, thus, ve...",email address is correct but I am not Carol St...
1,1,0.8547,"employee, write, enron, bankruptcy, fund, basi...",Connie Ballon-Almanza7215 Glen Trail DriveSan ...
2,2,0.9612,"com, enron, hotmail, net, priority, complease,...","genia.fitzgerald@enron.com, christi.nicolay@en..."
3,3,0.5730,"thank, call, know, need, question, send, let, ...","Theresa,Thanks. I appreciate it.Happy Thanksgi..."
4,5,0.6326,"click, way, sport, free, online, visit, specia...",Southwest Airlines Click 'n Save(R) E-mail Upd...
5,6,0.2713,"book, volume, check, head, counterparty, light...","You're probably right, Sharen, since the count..."
6,7,0.3882,"say, power, energy, state, electricity, utilit...","j..kean@enron.com, d..steffes@enron.com, linda..."
7,9,0.5184,"change, update, contract, start, schedule, opt...","Gerald,Both Chris and I are OK with the change..."
8,11,0.9435,"ect, hou, enronxgate, enron, lon, communicatio...","Cheryl Johnson@ENRONAllen/HOU/ECT@ECT, Bridget..."
9,12,0.3042,"agreement, document, file, appropriate, catch,...",Any problems/comments?AM ---------------------...


In [ ]:
#df_dominant_topic.to_csv('/content/drive/MyDrive/Colab Notebooks/data/dominant_topic.csv', sep=',')

In [ ]:
#sent_topics_sorteddf_mallet.to_csv('/content/drive/MyDrive/Colab Notebooks/data/sorted_df_sent_topics.csv', sep=',')

In [ ]:
df_dominant_topic['Keywords'] = df_dominant_topic['Keywords'].str.lower().str.split(',')

In [ ]:
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,21,0.3000,"[get, go, time, good, see, think, week, ...","It would be nice if you could be at my dinner,..."
1,1,9,0.2333,"[change, update, contract, start, schedule...",Absolutely.Good point! Can Peter start to dra...
2,2,21,0.5707,"[get, go, time, good, see, think, week, ...",My apologies. My schedule melted down after w...
3,3,9,0.4103,"[change, update, contract, start, schedule...","Vince,UK VAR breached the limit last week.UK t..."
4,4,12,0.3042,"[agreement, document, file, appropriate, c...",Any problems/comments?AM ---------------------...
5,5,19,0.4559,"[market, company, year, business, plan, a...",I certainly agree on the tip of the iceberg po...
6,6,3,0.1738,"[thank, call, know, need, question, send,...",Kim can you also invite Mike Roberts.
7,7,18,0.2963,"[site, notice, friend, link, win, page, ...",==============================================...
8,8,2,0.4819,"[com, enron, hotmail, net, priority, comp...","dporter3@enron.com, eric.linder@enron.com, gei..."
9,9,21,0.2289,"[get, go, time, good, see, think, week, ...",you still at school? call me at work.


In [ ]:
df_test = df_dominant_topic

In [ ]:
df_test['Keywords'] = df_test['Keywords'].apply(lambda keywords: [word.strip() for word in keywords])


In [ ]:
#df_test['Keywords'].head(10)

0    [get, go, time, good, see, think, week, day, m...
1    [change, update, contract, start, schedule, op...
2    [get, go, time, good, see, think, week, day, m...
3    [change, update, contract, start, schedule, op...
4    [agreement, document, file, appropriate, catch...
5    [market, company, year, business, plan, also, ...
6    [thank, call, know, need, question, send, let,...
7    [site, notice, friend, link, win, page, publis...
8    [com, enron, hotmail, net, priority, complease...
9    [get, go, time, good, see, think, week, day, m...
Name: Keywords, dtype: object

In [ ]:
# Function to calculate a score based on the number of matched words
def calculate_score(keywords, user_query_tokens):
   # matching_words = [word for word in user_query_tokens if word in keywords]
    matching_words = [word for word in user_query_tokens if word in keywords]
    return len(matching_words)

In [ ]:
user_query = "mail price rate"
user_query

'mail price rate'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Assuming 'user_query' is the user's query
#from nltk.tokenize import word_tokenize
tokenized_query = [token.lemma_ for token in nlp(user_query.lower())]

In [ ]:
print(tokenized_query)

['mail', 'price', 'rate']


In [ ]:
df_dominant_topic.columns

Index(['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords',
       'Text'],
      dtype='object')

In [ ]:
df_dominant_topic['Keywords'][0]

['get',
 ' go',
 ' time',
 ' good',
 ' see',
 ' think',
 ' week',
 ' day',
 ' make',
 ' take']

In [ ]:
df_test['Score'] = df_test['Keywords'].apply(lambda keywords: calculate_score(keywords, tokenized_query))

In [ ]:
df_test.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Score
0,0,21,0.3000,"[get, go, time, good, see, think, week, day, m...","It would be nice if you could be at my dinner,...",0
1,1,9,0.2333,"[change, update, contract, start, schedule, op...",Absolutely.Good point! Can Peter start to dra...,0
2,2,21,0.5707,"[get, go, time, good, see, think, week, day, m...",My apologies. My schedule melted down after w...,0
3,3,9,0.4103,"[change, update, contract, start, schedule, op...","Vince,UK VAR breached the limit last week.UK t...",0
4,4,12,0.3042,"[agreement, document, file, appropriate, catch...",Any problems/comments?AM ---------------------...,0
5,5,19,0.4559,"[market, company, year, business, plan, also, ...",I certainly agree on the tip of the iceberg po...,1
6,6,3,0.1738,"[thank, call, know, need, question, send, let,...",Kim can you also invite Mike Roberts.,0
7,7,18,0.2963,"[site, notice, friend, link, win, page, publis...",==============================================...,0
8,8,2,0.4819,"[com, enron, hotmail, net, priority, complease...","dporter3@enron.com, eric.linder@enron.com, gei...",0
9,9,21,0.2289,"[get, go, time, good, see, think, week, day, m...",you still at school? call me at work.,0


In [ ]:
df_test[df_test['Score'] == 3 ]

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Score
75,75,14,0.1832,"[mail, price, rate, transaction, section, amou...",T Paul Johnson@ENRONA number of pieces of on g...,3
134,134,14,0.2214,"[mail, price, rate, transaction, section, amou...",rich.heidorn@ipgdirect.comAug.-Nov.marjorie.ad...,3
176,176,14,0.1789,"[mail, price, rate, transaction, section, amou...",Markets and 21 December 2001 Day-Ahead MarketT...,3
184,184,14,0.2194,"[mail, price, rate, transaction, section, amou...",BUY 25 OPVSPT at 83.0000 - SALT RIVER PROJE...,3
677,677,14,0.2159,"[mail, price, rate, transaction, section, amou...","Dawn,NGI-SOCALNGI-PGE/CGDidn't you used to wor...",3
935,935,14,0.3543,"[mail, price, rate, transaction, section, amou...",---------------------- Forwarded by Evelyn Met...,3
1074,1074,14,0.1630,"[mail, price, rate, transaction, section, amou...","As of today, Power Gas Marketing volume has de...",3
1142,1142,14,0.3180,"[mail, price, rate, transaction, section, amou...","No, I think this one is for you...Marcus Nette...",3
1315,1315,14,0.2686,"[mail, price, rate, transaction, section, amou...",Mike Swerzbindeal 517384Prebon shows price as ...,3
1334,1334,14,0.3132,"[mail, price, rate, transaction, section, amou...",---------------------- Forwarded by Darron C G...,3


In [ ]:
relevant_emails = df_test[df_test['Score']>=2]

In [ ]:
relevant_emails

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Score
75,75,14,0.1832,"[mail, price, rate, transaction, section, amou...",T Paul Johnson@ENRONA number of pieces of on g...,3
134,134,14,0.2214,"[mail, price, rate, transaction, section, amou...",rich.heidorn@ipgdirect.comAug.-Nov.marjorie.ad...,3
176,176,14,0.1789,"[mail, price, rate, transaction, section, amou...",Markets and 21 December 2001 Day-Ahead MarketT...,3
184,184,14,0.2194,"[mail, price, rate, transaction, section, amou...",BUY 25 OPVSPT at 83.0000 - SALT RIVER PROJE...,3
677,677,14,0.2159,"[mail, price, rate, transaction, section, amou...","Dawn,NGI-SOCALNGI-PGE/CGDidn't you used to wor...",3
935,935,14,0.3543,"[mail, price, rate, transaction, section, amou...",---------------------- Forwarded by Evelyn Met...,3
1074,1074,14,0.1630,"[mail, price, rate, transaction, section, amou...","As of today, Power Gas Marketing volume has de...",3
1142,1142,14,0.3180,"[mail, price, rate, transaction, section, amou...","No, I think this one is for you...Marcus Nette...",3
1315,1315,14,0.2686,"[mail, price, rate, transaction, section, amou...",Mike Swerzbindeal 517384Prebon shows price as ...,3
1334,1334,14,0.3132,"[mail, price, rate, transaction, section, amou...",---------------------- Forwarded by Darron C G...,3


In [ ]:
len(relevant_emails)

48

In [ ]:
!pip install rank-bm25

In [ ]:
from rank_bm25 import BM25Okapi
# Tokenize and preprocess the 'Text' column
tokenized_text = [text.split() for text in relevant_emails['Text']]

In [ ]:
tokenized_text

[['T',
  'Paul',
  'Johnson@ENRONA',
  'number',
  'of',
  'pieces',
  'of',
  'on',
  'going',
  'litigation',
  'and',
  'arbitration',
  'being',
  'conducted',
  'onbehalf',
  'of',
  'EML',
  'and',
  'EMC',
  'have',
  'very',
  'recently',
  'come',
  'to',
  'our',
  'attention',
  'which',
  'wesummarise',
  'below.',
  'We',
  'have',
  'endeavoured',
  'to',
  'establish',
  'the',
  'detail',
  'of',
  'these',
  'casesas',
  'quickly',
  'as',
  'possible,',
  'but',
  'in',
  'some',
  'instances',
  'await',
  'information',
  'from',
  'outsidecounsel',
  'involved.',
  'The',
  'summary',
  'is',
  'therefore',
  'necessarily',
  'provisional,',
  'and',
  'willneed',
  'to',
  'be',
  'supplemented.',
  'There',
  'are',
  'in',
  'addition',
  'other',
  'claims',
  'of',
  'which',
  'we',
  'havebeen',
  'made',
  'aware,',
  'largely',
  'being',
  'run',
  'by',
  'MG',
  'Hong',
  'Kong',
  'and',
  'Beijing',
  'offices.Details',
  'are',
  'presently',
  'so',

In [ ]:
bm25 = BM25Okapi(tokenized_text)

In [ ]:
# Assuming 'tokenized_query' is the tokenized user query
bm25_scores = bm25.get_scores(tokenized_query)
bm25_scores

array([0.        , 0.        , 0.        , 0.30247387, 0.        ,
       0.        , 0.        , 0.47130902, 0.65187941, 0.        ,
       0.        , 0.        , 0.55002525, 0.48655181, 0.        ,
       0.        , 0.        , 0.        , 0.61147205, 0.18897361,
       0.        , 0.46163399, 2.5804324 , 7.0363374 , 4.11096644,
       0.4690405 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.60250616, 0.        , 0.        , 4.17243756,
       0.        , 2.64558357, 0.        , 0.50956467, 0.58974413,
       0.60183108, 0.        , 0.47436807, 0.23299673, 0.        ,
       0.        , 0.50336361, 0.59325339])

In [ ]:
# Combine BM25 scores with relevance scores from LDA
final_scores = [bm25_score + 1 if index in relevant_emails else bm25_score for index, bm25_score in enumerate(bm25_scores)]

In [ ]:
# Rank emails based on final scores
ranked_emails = sorted(range(len(final_scores)), key=lambda i: final_scores[i], reverse=True)

In [ ]:
# Print the ranked emails
for i in ranked_emails:
    print(f"Email {i + 1}: Final Score = {final_scores[i]}")

Email 24: Final Score = 7.036337398610622
Email 35: Final Score = 4.172437556944025
Email 25: Final Score = 4.110966436055385
Email 37: Final Score = 2.645583569772919
Email 23: Final Score = 2.580432398511323
Email 9: Final Score = 0.6518794112368398
Email 19: Final Score = 0.6114720486360704
Email 32: Final Score = 0.6025061635721823
Email 41: Final Score = 0.6018310812095371
Email 48: Final Score = 0.5932533948944985
Email 40: Final Score = 0.5897441262547473
Email 13: Final Score = 0.5500252494321223
Email 39: Final Score = 0.5095646664598157
Email 47: Final Score = 0.5033636149632148
Email 14: Final Score = 0.48655181214555865
Email 43: Final Score = 0.4743680738345595
Email 8: Final Score = 0.47130902232310684
Email 26: Final Score = 0.4690405005811656
Email 22: Final Score = 0.4616339888522747
Email 4: Final Score = 0.30247387349543614
Email 44: Final Score = 0.23299672777463348
Email 20: Final Score = 0.18897361410970115
Email 1: Final Score = 0.0
Email 2: Final Score = 0.0
Ema

In [ ]:
# Create a DataFrame with Document_No, Score, and Text
result_df = pd.DataFrame({
    "Document_No": relevant_emails["Document_No"],
    "Score": bm25_scores,
    "Text": relevant_emails["Text"]
})

# Sort the DataFrame by BM25 score in descending order
result_df = result_df.sort_values(by="Score", ascending=False)

# Print the ranked documents
for rank, row in result_df.iterrows():
    print(f"Rank {rank + 1}: Score={row['Score']:.4f}, Document: {row['Text']}")

Rank 5612: Score=7.0363, Document: Rebecca,My mail stop is EB 3125A. I e-mailed you that info. on Oct.30 and left you avoice mail message.CD
Rank 7519: Score=4.1724, Document: Julian and I only have one change.  We would like to have the full text tothe second paragraph of the Floating Price difinition included so thecustomer sees it up front.  This is the language which I pulled from theThe parties agree that the Floating Amounts payable under this Transactionshall be made in US Dollars, and the rate of exchange of Argentine Pesos toUS Dollars shall be of one (1) Argentine Peso per US Dollar as set forthunder Law 23,928, as amended, while there is free convertibility of theArgentine Peso and the rate of exchange therein set forth is of one (1)Argentine Peso per US Dollar. Otherwise, the rate of exchange betweenArgentine Pesos and US Dollars shall be calculated using the exchange rate,sell, for transfers abroad established by Banco Naci?n Argentina at theclosing of the Business Day pri

In [ ]:
result_df.Text.iloc[2]

'Your 22-24 product is product #60987. Your Oct01 # is 60989.  Please check them and make sure they are exactly what you want to trade.Thanks,Melba Lozano713-345-8986713-646-8511 Fax713-698-4847 Pager-----Original Message-----How about "LaMaquina-I/B Link"-----Original Message-----The abbreviation can only be 21 characters long.  Please let me know how to modify it.-----Original Message-----Can this be set up to start this weekend, the 9/22?  I will build this product combo for daily, weekly, and definitely monthly.  LaMachina is misspelled (my mistake); the correct spelling should be LaMaquina.  Can the abbreviation for the web be "LaMaquina-IgnacioBlanco"  Thanks!-----Original Message-----Michelle,What Dates do you need set up?  Does the abbreviation for the web look o.k?US PLCapTW Frm   WFS-IgnacioBlanco (abbreviation)This Transaction provides firm pipeline transportation capacity from the primary receipt point of WFS LaMachina [500539] to the primary delivery point of Ignacio/Blanc